### Bunch of imports, will see later

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
#Sklearn
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE

#other
import concurrent.futures
import time
import pyLDAvis.sklearn
from pylab import bone, pcolor, colorbar, plot, show, rcParams, savefig
import warnings
warnings.filterwarnings('ignore')


%matplotlib inline

# Plotly 
from plotly import tools
import chart_studio.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.figure_factory as ff

#spaCy
import spacy
nlp = spacy.load("fr_core_news_lg")
from spacy.lang.fr.stop_words import STOP_WORDS
from spacy.lang.fr import French
import string
punctuations = string.punctuation
stopwords = list(STOP_WORDS)

In [6]:
#df=pd.read_csv('../../data/Cataloguev1.csv', sep=';',error_bad_lines=False, encoding='latin-1') #le fichier original
df2=pd.read_csv('../../data/Catalogue.csv', sep=';',error_bad_lines=False, encoding='latin-1') #le fichier original

In [17]:
id_table=list(df['id'])

In [18]:
id_table=np.array(id_table, dtype=object)

In [19]:
id_table

array(['5ef6c4b8cb925cf05ddd876a', '5ef69a135ef68f28bfdd876a',
       '5ef69a1328a6ea1e97dd876a', ..., '5da117af06e3e7269429a310',
       '5cd5b6169ce2e7777eef0f2d', '5e3b5c2edee7e7777ff5c8ee'],
      dtype=object)

In [20]:
np.save('../../data/id_table', id_table, allow_pickle=True )

In [4]:
df1=df[['Column2', 'Column4', 'minuscules','minuscules.1']]
df1=df1.astype(str)

In [5]:
df1['sentences']=df1[['Column2', 'Column4', 'minuscules','minuscules.1']].agg('. '.join, axis=1)
df1['nodesc']=df1[['Column2', 'minuscules.1']].agg('. '.join, axis=1)

In [6]:
df1['nodesc'][23181]

"périmètre d'exposition au risque du pprn de la commune de plombières-lès-dijon en côte-d'or. donnees-ouvertes,geoscientific-information,inondation,passerelle-inspire,risque-zonages-risque-naturel"

In [7]:
import re

cp_1252_chars = {
    # from http://www.microsoft.com/typography/unicode/1252.htm
    u"\x80": u"\u20AC", # EURO SIGN
    u"\x82": u"\u201A", # SINGLE LOW-9 QUOTATION MARK
    u"\x83": u"\u0192", # LATIN SMALL LETTER F WITH HOOK
    u"\x84": u"\u201E", # DOUBLE LOW-9 QUOTATION MARK
    u"\x85": u"\u2026", # HORIZONTAL ELLIPSIS
    u"\x86": u"\u2020", # DAGGER
    u"\x87": u"\u2021", # DOUBLE DAGGER
    u"\x88": u"\u02C6", # MODIFIER LETTER CIRCUMFLEX ACCENT
    u"\x89": u"\u2030", # PER MILLE SIGN
    u"\x8A": u"\u0160", # LATIN CAPITAL LETTER S WITH CARON
    u"\x8B": u"\u2039", # SINGLE LEFT-POINTING ANGLE QUOTATION MARK
    u"\x8C": u"\u0152", # LATIN CAPITAL LIGATURE OE
    u"\x8E": u"\u017D", # LATIN CAPITAL LETTER Z WITH CARON
    u"\x91": u"\u2018", # LEFT SINGLE QUOTATION MARK
    u"\x92": u"\u2019", # RIGHT SINGLE QUOTATION MARK
    u"\x93": u"\u201C", # LEFT DOUBLE QUOTATION MARK
    u"\x94": u"\u201D", # RIGHT DOUBLE QUOTATION MARK
    u"\x95": u"\u2022", # BULLET
    u"\x96": u"\u2013", # EN DASH
    u"\x97": u"\u2014", # EM DASH
    u"\x98": u"\u02DC", # SMALL TILDE
    u"\x99": u"\u2122", # TRADE MARK SIGN
    u"\x9A": u"\u0161", # LATIN SMALL LETTER S WITH CARON
    u"\x9B": u"\u203A", # SINGLE RIGHT-POINTING ANGLE QUOTATION MARK
    u"\x9C": u"\u0153", # LATIN SMALL LIGATURE OE
    u"\x9E": u"\u017E", # LATIN SMALL LETTER Z WITH CARON
    u"\x9F": u"\u0178", # LATIN CAPITAL LETTER Y WITH DIAERESIS
} 

def fix_1252_codes(text):
    """
    Replace non-standard Microsoft character codes from the Windows-1252 character set in a unicode string with proper unicode codes.
    Code originally from: http://effbot.org/zone/unicode-gremlins.htm
    """
    if re.search(u"[\x80-\x9f]", text):
        def fixup(m):
            s = m.group(0)
            return cp_1252_chars.get(s, s)
        if isinstance(text, type("")):
            text = str(text)
        text = re.sub(u"[\x80-\x9f]", fixup, text)
    return text 

def super_cleaning(dfx):
    for i in range(len(dfx)):
        dfx[i]=fix_1252_codes(dfx[i])
        dfx[i]=re.sub(r'\S*.(fr|org|com|map|jpeg|jpg|zip|png)\b','', dfx[i])
        dfx[i]=re.sub(r"""
               [,;@#/\\?*[\]\r\n\<\>\_\{\}\»\«\\\(\)!&$]+  # Accept one or more copies of punctuation
               \ *           # plus zero or more copies of a space,
               """,
               " ",          # and replace it with a single space
               dfx[i], flags=re.VERBOSE)
        
        dfx[i]=re.sub(r'\\x\w+',' ', dfx[i])
        dfx[i]=re.sub(r'\bx\w+', '', dfx[i])
        dfx[i]=re.sub(r'\S+\d+\w+', " ", dfx[i])
        dfx[i]=re.sub(r'(?:^| )\w(?:$| )',' ',dfx[i])
        dfx[i]=re.sub(' +', ' ', dfx[i])
       
     #\b\x\w+
    return(dfx)

#LINKS
#text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
#REPLACE - with space 
#text= text.replace('-', ' ')
#REPLACE \x92 with ' 


In [238]:
#cleaned1=super_cleaning(df1['sentences'])
nodesc_clean=super_cleaning(df1['nodesc'])

In [240]:
nodesc_clean[i]

'p.l.u. de capvern. nan'

In [8]:
cleaned2=super_cleaning(df1['sentences'])

In [263]:
i=random.randint(1,len(nodesc_clean))
nodesc_clean[i]

'secteur des cartes communales en seine et marne. donnees ouvertes passerelle inspire planning cadastre usage des sols'

In [9]:
cleaned2[20751][:200]

'zonage réglementaire du plan de prévention du risque inondation par débordement de la rivière marne de châlons-en-champagne – secteur amont. Direction Départementale des Territoires de la Marne. pour '

In [10]:
cleaned2.to_pickle('../../data/cleaned2.pkl')
cleaned2_np=cleaned2.to_numpy()
np.save('../../data/cleaned2.npy', cleaned2_np, allow_pickle=True, fix_imports=True)

In [264]:
nodesc_clean.to_pickle('../../data/nodesc_clean.pkl')
nodesc_clean_np=nodesc_clean.to_numpy()
np.save('../../data/nodesc_clean.npy', nodesc_clean_np, allow_pickle=True, fix_imports=True)

### Liste des producteurs de données

In [11]:
producteurs=set(df1['Column4'])

In [14]:
import collections
counter=collections.Counter(df1['Column4'])
print(counter)

Counter({'Ressourcerie datalocale': 3176, 'nan': 2551, 'Région Île-de-France': 1249, 'Direction Départementale des Territoires et de la Mer de la Somme': 1029, 'Direction Départementale des Territoires du Cher': 1017, "Direction Départementale des Territoires de l'Ariège": 909, 'Région Occitanie / Pyrénées Méditerranée': 882, 'Mairie de Paris': 659, "Ministère de l'Intérieur": 613, "Communauté d'Agglomération du Saint-Quentinois": 602, 'Collectivité de Corse': 599, 'Toulouse métropole': 591, 'Direction Départementale des Territoires et de la Mer des Pyrénées-Atlantiques': 589, "Direction Départementale des Territoires de l'Aube": 580, 'Direction Départementale des Territoires de Haute-Savoie': 553, "Système d'Information sur l'Eau": 534, 'Métropole de Lyon': 517, 'Direction Départementale des Territoires de Meurthe-et-Moselle': 444, 'NosDonnées.fr': 427, "Ministère de l'économie et des finances": 391, "DDT Côte-d'Or": 390, 'La Rochelle': 379, 'DDT Tarn-et-Garonne': 355, 'DDTM Gard': 35

In [17]:
producteurs_pd=pd.DataFrame(producteurs)

In [18]:
producteurs_pd.to_pickle('../../data/prod_set.pkl')

In [19]:
df_prod=df[['Column4', 'minuscules.1']]

In [28]:
df_prod=df_prod.astype('str')

In [30]:
df_prod=df_prod.groupby(['Column4'], as_index = False).agg({'minuscules.1': ' '.join})

In [43]:
df_prod_np=df_prod.to_numpy()

In [47]:
df_prod.to_pickle('../../data/df_prod_tags.pkl')

In [45]:
df_prod_np[25]

array(["Agence Nationale d'Appui à la Performance des établissements de santé et médico-sociaux (ANAP)",
       'clinique,comparaison,etablissement-de-sante,finances,hopital,hospidiag,indicateurs,medecine,medecine-chirurgie-obstetrique,organisation-des-soins,performance,pratiques-professionnelles,qualite-des-soins,ressources-humaines'],
      dtype=object)

In [ ]:
np.load('.')

#### Merge with locs col

In [4]:
locs=pd.read_pickle('../Scripts/FOLDER/locs.pkl')

In [11]:
df2.append(locs['ncc'])

MemoryError: Unable to allocate 13.7 GiB for an array with shape (42837, 42837) and data type float64

In [13]:
df2['pred locs']=locs['ncc']

In [15]:
df2.to_csv('../../data/Catalogue_locs.csv')

In [18]:
df2

,id,title,slug,organization,description,spatial.granularity,spatial.zones,tags,Unnamed: 8,Unnamed: 9,...,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,pred locs
0,5ef6c4b8cb925cf05ddd876a,Laboratoires d'analyses médicales VIHTEST,laboratoires-danalyses-medicales-vihtest,Mairie de Paris,**Jeu de données localisant les Laboratoire d'...,NaN,NaN,"depistage,equipements,sante,services,social,vih",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PARIS
1,5ef69a135ef68f28bfdd876a,2020-2026_Elus Grand Chambéry,2020-2026-elus-grand-chambery,Grand Chambéry,Ce jeu de données présente les **conseillers c...,NaN,NaN,"administration,citoyennete,election,elu-e-s,ep...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHAMBERY
2,5ef69a1328a6ea1e97dd876a,2020-2026_Elus communaux Grand Chambéry,2020-2026-elus-communaux-grand-chambery,Grand Chambéry,Ce jeu de données présente les conseillers mun...,NaN,NaN,"administration,citoyennete,commune,election,el...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHAMBERY
3,5ef68e8220f90f5d8ddd876a,Mon reseau mobile ARCEP - Zones blanches 4G da...,mon-reseau-mobile-arcep-zones-blanches-4g-data...,Collectivité de Corse,Ce jeu de données provient d'un service public...,NaN,NaN,"infrastructures,telephone-mobile",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CORSE
4,5ef68e82714cc9b811dd876a,Mon reseau mobile ARCEP - Zones blanches 3G da...,mon-reseau-mobile-arcep-zones-blanches-3g-data...,Collectivité de Corse,Ce jeu de données provient d'un service public...,NaN,NaN,"infrastructures,telephone-mobile",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CORSE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42832,5ce66e0d06e3e76b9e62521e,Consommation d'eau par compteur à Digne-les-Bains,consommation-deau-par-compteur-a-digne-les-bains,Ville de Digne-les-Bains,Relevé détaillé de la consommation d'eau à Dig...,NaN,NaN,"canalisation,consommation,distribution,eau,reseau",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DIGNE LES BAINS
42833,5ce889a406e3e76c3ed57ecd,Consommation eau potable par quartier à Digne-...,consommation-eau-potable-par-quartier-a-digne-...,Ville de Digne-les-Bains,Rapport de diagnostic de sectorisation du rése...,NaN,NaN,"canalisation,consommation,distribution,eau,reseau",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DIGNE LES BAINS
42834,5da117af06e3e7269429a310,Filaire de voies sur le territoire de la commu...,fr-241300417-adr-accm-filaire,Arles Crau Camargue Montagnette,Représentation de l'axe des voies qui porte le...,NaN,NaN,"axe,donnees-ouvertes,filaire,localisation,nom,...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ARLES
42835,5cd5b6169ce2e7777eef0f2d,Filaire des voies - ACCM,filaire-des-voies-accm,Arles Crau Camargue Montagnette,Représentation des axes de voies sur l'ensembl...,NaN,NaN,"axe-de-voie,filaire-de-voie,localisation,nom-d...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ARLES


In [21]:
df2.loc[37778]['description']

'Toutes les délibérations de la CNIL depuis l\'origine, la première délibération ayant été rendue en 1979. \n\nLeurs modalités de publication sont définies par la loi Informatique et Libertés, par son décret d\x92application ainsi que par le règlement intérieur de la Commission. \n\nLes délibérations adoptées entre le 1er janvier 1979 et le 3 mai 2012 ont été publiées dans leur version mise à jour au plus tard le 4 mai 2012. Les délibérations adoptées postérieurement au 3 mai 2012 portent mention de leur date de publication effective. \n\nLes délibérations de la CNIL sont également accessibles sur <a href="http://legifrance.gouv.fr/initRechExpCnil.do" target="_blank">Legifrance</a>.\n\n'